In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization


2022-11-14 12:30:22.508960: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 12:30:22.736674: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset ='/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Kerasmodel/aclImdb'

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['imdbEr.txt', 'README', 'train', 'imdb.vocab', 'test']

In [3]:
train_dir = '/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Teamwise_dataset'
os.listdir(train_dir)

['1']

In [4]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/turning/Desktop/CL_project/CL_PROJECT_CODE/Teamwise_dataset/unsup'

In [5]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 21668 files belonging to 2 classes.
Using 17335 files for training.


2022-11-14 12:30:31.580361: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 21668 files belonging to 2 classes.
Using 4333 files for validation.


In [6]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

0 b'Okay. Who was it? Who gave Revolver 10 out of 10? Are you tripping of your head on Ecstasy pipes? There were so many of you. Did you do it for a dare? Is this some kind of cult? Or did Guy Richie himself sign up 788 times under different names?<br /><br />Before I say anything else, I\'ll say this. Just because you don\'t understand a film doesn\'t mean that it\'s not great. Maybe you\'ve had a bad day at work, or you sat down to watch a film after you had a row with your wife and then weren\'t in the mood. Maybe there\'s a more fundamental stumbling block- like you just don\'t have the mental capacity or a highly enough developed philosophical sense to engage with it. BUT. And this is a very, very big but. The XXL elephant-sized mega-but to end all buts.<br /><br />PLEASE don\'t confuse incoherence for complexity, and please don\'t confuse this two hour non-squirter for an interesting film. Really. You may think you are pretty smart. You may even think of yourself as somewhat of a

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [10]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[ 0.00640338,  0.02891463,  0.03818734,  0.01890494,  0.00337709],
       [ 0.04226183,  0.01441587,  0.03041745, -0.03848713,  0.03746002],
       [ 0.01426921,  0.03423483, -0.01242656, -0.01156135, -0.00521504]],
      dtype=float32)

In [11]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

In [12]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [13]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [14]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
17/17 [==============================] - 3s 126ms/step - loss: 0.6874 - accuracy: 0.5803 - val_loss: 0.6842 - val_accuracy: 0.5634
Epoch 2/15
17/17 [==============================] - 2s 91ms/step - loss: 0.6775 - accuracy: 0.5803 - val_loss: 0.6783 - val_accuracy: 0.5634
Epoch 3/15
17/17 [==============================] - 2s 93ms/step - loss: 0.6698 - accuracy: 0.5803 - val_loss: 0.6715 - val_accuracy: 0.5634
Epoch 4/15
17/17 [==============================] - 2s 90ms/step - loss: 0.6596 - accuracy: 0.5803 - val_loss: 0.6596 - val_accuracy: 0.5634
Epoch 5/15
17/17 [==============================] - 2s 93ms/step - loss: 0.6456 - accuracy: 0.5803 - val_loss: 0.6444 - val_accuracy: 0.5634
Epoch 6/15
17/17 [==============================] - 2s 96ms/step - loss: 0.6273 - accuracy: 0.5803 - val_loss: 0.6259 - val_accuracy: 0.5634
Epoch 7/15
17/17 [==============================] - 2s 95ms/step - loss: 0.6050 - accuracy: 0.5803 - val_loss: 0.6043 - val_accuracy: 0.5634
Epoch 8/15
1

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trai

In [22]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 5509), started 0:00:14 ago. (Use '!kill 5509' to kill it.)

In [19]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [20]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [21]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass